In [17]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import sqlite3

In [19]:
# 스타벅스 매장 지도 페이지 URL
url = "https://www.starbucks.co.kr/store/store_map.do"

# 크롬 드라이버 실행
driver = webdriver.Chrome()
driver.get(url)

# 페이지 로딩 대기
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "header.loca_search > h3 > a")))

# 시/도 리스트 가져오기
sidos = []
element = driver.find_element(By.CSS_SELECTOR, "header.loca_search > h3 > a")
element.click()
time.sleep(1)

elements = driver.find_elements(By.CSS_SELECTOR, "div.loca_step1_cont > ul > li > a")
for element in elements:
    sidos.append(element.text)

# 결과 저장할 딕셔너리
store_data = {}

# 각 시/도별 매장 정보 크롤링
for sido in sidos:
    driver.get(url)
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "header.loca_search > h3 > a")))

    element = driver.find_element(By.CSS_SELECTOR, "header.loca_search > h3 > a")
    element.click()

    # 시/도 선택
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.LINK_TEXT, sido)))
    element = driver.find_element(By.LINK_TEXT, sido)
    element.click()

    # "전체" 버튼 클릭 (세종 제외)
    if sido != "세종":
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.LINK_TEXT, "전체")))
        driver.find_element(By.LINK_TEXT, "전체").click()

    # 매장 목록 가져오기
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#mCSB_3_container > ul > li")))
    store_list = driver.find_elements(By.CSS_SELECTOR, "#mCSB_3_container > ul > li")

    temp_list = []
    for count, store in enumerate(store_list, start=1):
        lat = store.get_attribute("data-lat")
        long = store.get_attribute("data-long")
        name = store.get_attribute("data-name")
        storeinfo = store.find_element(By.TAG_NAME, "p").text.split("\n")

        address = storeinfo[0] if len(storeinfo) > 0 else "-"
        phone_number = storeinfo[1] if len(storeinfo) > 1 else "-"

        temp_list.append((name, lat, long, address, phone_number))

        # 스크롤 조정 (3개마다 스크롤)
        if count % 3 == 0 and count < len(store_list):
            driver.execute_script(
                "var su=arguments[0];\
                var dom=document.querySelectorAll('#mCSB_3_container > ul > li')[su];\
                dom.scrollIntoView();", count
            )
        time.sleep(0.5)  # 스크롤 후 잠깐 대기

    store_data[sido] = temp_list

# 크롬 드라이버 종료
driver.quit()

# 결과 출력 (확인용)
for sido, stores in store_data.items():
    print(f"\n[{sido}]")
    for store in stores:
        print(store)

NameError: name 'WebDriverWait' is not defined

In [18]:
#스타벅스 >지역검색 > 전체 > 매장,주소,전화번호 출력 > db저장하기

url = 'https://www.starbucks.co.kr/store/store_map.do'
sido = []


driver = webdriver.Chrome()
driver.get(url)

time.sleep(1)
elements = driver.find_elements("css selector", "div.loca_step1_cont > ul > li > a")
print(len(elements))


#'지역 검색' 클릭
local_search = driver.find_element('link text','지역 검색')
local_search.click()
time.sleep(1)

#각 지역을 순회하여 매장 정보 수집
region = driver.find_element('css selector',"div.loca_step1_cont > ul > li > a")
all_store_info=[]

for region_element in region:
    region_name = region_element.text
    print(f"Processing region: {region_name}")
    
    region_element.click()
    time.sleep(1)

#'전체' 클릭
region_search = driver.find_element('link text','전체')
region_search.click()
time.sleep(1)

#매장,주소,전화번호 출력
store_list = driver.find_elements("css selector", "#mCSB_3_container > ul > li")
for store in store_list:
    store_lat = store.get_attribute('data-lat') #위도
    store_long = store.get_attribute('data-long') #경도
    store_name = store.get_attribute('data-name')
    store_info = store.find_element('p')
    arr_store_info = store_info.text.split("\n")
    if len(arr_store_info)==2:
        addr = arr_store_info[0]
        phone_number = arr_store_info[1]
    else:
        addr='-'
        phone_number='-'

    all_store_info.append((store_name, store_lat, store_long, addr, phone_number)) 

#'지역 검색'으로 돌아가기
local_search = driver.find_element('link text','지역 검색')
local_search.click()
time.sleep(1)

#모든 매장 정보 출력
for store_info in all_store_info:
    print(store_info)

driver.quit()

0


ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:void(0);">...</a> is not clickable at point (245, 277). Other element would receive the click: <div class="loading_dimm" style="z-index: 20000; display: block;"></div>
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00007FF6BC4FFE45+26629]
	(No symbol) [0x00007FF6BC466010]
	(No symbol) [0x00007FF6BC2F931A]
	(No symbol) [0x00007FF6BC3577A9]
	(No symbol) [0x00007FF6BC355162]
	(No symbol) [0x00007FF6BC352201]
	(No symbol) [0x00007FF6BC351108]
	(No symbol) [0x00007FF6BC3428A4]
	(No symbol) [0x00007FF6BC377AAA]
	(No symbol) [0x00007FF6BC342156]
	(No symbol) [0x00007FF6BC377CC0]
	(No symbol) [0x00007FF6BC3A0169]
	(No symbol) [0x00007FF6BC377883]
	(No symbol) [0x00007FF6BC340550]
	(No symbol) [0x00007FF6BC341803]
	GetHandleVerifier [0x00007FF6BC8572BD+3529853]
	GetHandleVerifier [0x00007FF6BC86DA22+3621858]
	GetHandleVerifier [0x00007FF6BC8624D3+3575443]
	GetHandleVerifie

In [ ]:
#DB 저장하기